# import modules

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time
import os
import csv

# start a selenium driven browser

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')

web_url = 'https://www.calorieking.com/us/en/'
driver = webdriver.Chrome(chrome_options=options)
result_lst = []
target_brand =''
food_lst = []
target_food =''
target_food_url = ''
calories = ''
fat = ''
carbs = ''
fiber = ''
protein = ''
status = False

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [3]:
# get brand key word
def get_key_word():
    driver.get(web_url)
    time.sleep(0.2)
    search_terms = input('Please enter a search terms.\t')
    print(f'search for the terms: {search_terms}')
    # search the key words
    search_field = driver.find_element_by_id('keywords')
    search_field.send_keys(search_terms)
    search_field.send_keys(Keys.RETURN)
    # get search terms results
    brands = driver.find_elements_by_class_name('jss5')
    global result_lst
    result_lst = [ i.find_element_by_class_name('MuiTypography-root').text for i in brands]
    if 'for iOS'in str(result_lst):
        print('Please try another search tearms for brand.')
        get_key_word()

In [4]:
#narrow down to the specific brand
def get_brand():
    def check_brand():
        for i in result_lst:
            yield(i)
    brands = check_brand()
    correct = 'F'
    global target_brand
    global status
    status = False
    while correct == 'F':
        try:
            brand_candidate = next(brands)
            correct = input(f'If {brand_candidate} the brand you are looking for? T/F:\t').upper()
            if correct =='T':
                target_brand = str(brand_candidate)
                status = True
                break
        except: 
            print('Please try another search tearms for brand.')
            get_key_word()
            get_brand()
            break
        
            


In [5]:
# search for the dish of the brand found in get_brand()
def get_key_word_checked():
    global target_brand
    driver.get(web_url)
    time.sleep(0.2)
    meal_terms = input('Please enter the main meal ingrediant.\t').upper()
    search_terms = target_brand + ' ' + meal_terms
    print(f'search for the terms: {search_terms}')
    # search the key words
    search_field = driver.find_element_by_id('keywords')
    search_field.send_keys(search_terms)
    search_field.send_keys(Keys.RETURN)
    # get search terms results
    foods = driver.find_elements_by_class_name('jss374')
    global food_lst
    food_lst = [ i.text for i in foods]
    if 'for iOS'in str(food_lst):
        print('Please try another search tearms for brand.')
        get_key_word_checked()


In [6]:
# narrow down to the specific dish and get the dish's info link
def get_food():
    global target_food_url
    count=0
    def check_food():
        for i in food_lst:
            yield(i)
    foods = check_food()
    correct = 'F'
    global target_food
    global status
    status = False
    while correct == 'F':
        try:
            food_candidate = next(foods)
            correct = input(f'If {food_candidate} the meal you are looking for? T/F:\t').upper()
            if correct =='T':
                target_food = str(food_candidate)
                url = driver.find_elements_by_class_name('MuiListItem-button')[count].get_attribute('href')
                target_food_url = (url)
                status = True
                break
            else:
                count=count+1
        except: 
            print('Please try another search tearms for meal.')
            get_key_word_checked()
            get_food()
            break
            
    


In [7]:
def get_food_info():
    
    global target_food_url
    driver.get(target_food_url)
    infos = driver.find_element_by_class_name('jss374')
    infos = infos.text.split('\n')
    
    global calories
    global fat
    global carbs
    global fiber
    global protein
    calories = infos[0].split(' ')[0]
    fat = infos[6].split(' ')[0]
    carbs = infos[8].split(' ')[0]
    fiber = infos[10].split(' ')[0]
    protein = infos[12].split(' ')[0]
    
    


In [8]:
meal_translator = {'B':'Breakfast','L':"Lunch",'D':'Dinner','O':"Others"}
def save_result():
    if os.path.isfile('food_recoder.csv') == False:
        with open('food_recoder.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Time', 'Meal_Time' , 'Brand', 'Food', 'Calories', 'Fat', 'Carbs', 'Fiber', 'Protein'])
    
    time = input('Please enter the date. mm/dd/yyyy.\t')
    print('Please enter the meal time.')
    meal_time = input('B---Breakfast\nL---Lunch\nD---Dinner\nO---Others').upper()
    meal_time_transfered = meal_translator[meal_time]
    with open('food_recoder.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([time, meal_time_transfered, target_brand, target_food, calories, fat, carbs, fiber, protein])



In [9]:
def run_again():
    run = input('Have another meal to record? T/F \t').upper()
    if run == 'T':
        main()
    else:
        print('Thanks for using this software.')
        

In [10]:
def main():
    get_key_word()
    get_brand()
    global status
    if status == True:
        get_key_word_checked()
    get_food()
    if status == True:
        get_food_info()
    save_result()
    run_again()

In [11]:
main()

Please enter a search terms.	popeye
search for the terms: popeye
If Popeye's the brand you are looking for? T/F:	t
Please enter the main meal ingrediant.	chicken
search for the terms: Popeye's CHICKEN
If Classic or Spicy Chicken Sandwich the meal you are looking for? T/F:	t
Please enter the date. mm/dd/yyyy.	01/02/2021
Please enter the meal time.
B---Breakfast
L---Lunch
D---Dinner
O---Othersb
Have another meal to record? T/F 	f
Thanks for using this software.


In [12]:
# check for results
import pandas as pd
record = pd.read_csv('food_recoder.csv')
record.head()

,Time,Meal_Time,Brand,Food,Calories,Fat,Carbs,Fiber,Protein
0,01/02/2021,Breakfast,Popeye's,Classic or Spicy Chicken Sandwich,700,42,50,2,28
